In [ ]:
!python train_tabnet.py --train_path s3://fireguarddata/data/preprocessed_data/train.csv --val_path s3://fireguarddata/data/preprocessed_data/val.csv --epochs 2 --batch_size 8192

Steps to Train Your Model Using SageMaker Estimator:
## Upload Your Training Script to Local directory
Make sure your training scripts are uploaded locally. SageMaker expects the training script to be located in a local directory, not to S3.

### Setting up Training Job 
### Upload Tabnet py script to s3

**uploading script to s3**
training jobs expect scripts saved locally and not in s3. 

In [19]:
!aws s3 cp train_tabnet_balanced.py s3://fireguarddata/scripts/train_tabnet_balanced.py

upload: ./train_tabnet_balanced.py to s3://fireguarddata/scripts/train_tabnet_balanced.py


In [ ]:
!aws s3 cp train_tabnet_focal.py s3://fireguarddata/scripts/train_tabnet_focal.py

In [20]:
#verify upload
!aws s3 ls s3://fireguarddata/scripts/

2025-03-30 05:04:51       2999 train_tabnet.py
2025-03-30 05:57:32       3803 train_tabnet_balanced.py


In [1]:
#list files in local directory
!ls -l


total 104
drwxr-xr-x 2 vanel vanel  4096 Jan 26 13:20 Data
drwxr-xr-x 2 vanel vanel  4096 Mar 24 02:42 cb_2022_us_state_20m
drwxr-xr-x 2 vanel vanel  4096 Apr  2 20:00 files-moved
drwxr-xr-x 3 vanel vanel  4096 Mar 24 00:35 mtbs_fod_pts_data
drwxr-xr-x 2 vanel vanel  4096 Apr  2 20:00 scripts
-rw-r--r-- 1 vanel vanel 85940 Apr  2 19:40 sdk_tabnet_train.ipynb


In [3]:
#Create the scripts directory if it doesn't already exist:
!mkdir -p scripts

In [30]:
#move script to the scripts folder
!mv train_tabnet_balanced.py ./scripts/

**Create the Training Job Using SageMaker Python SDK**
let's set up the sagemaker training job

In [4]:
ls -l ./scripts

total 4
-rw-r--r-- 1 vanel vanel 2999 Apr  2 19:40 train_tabnet.py


Run the Cell and Monitor
- The cell will initiate the SageMaker training job.
- You can monitor the training job in the SageMaker Console under Training > Training jobs.
- Once the job completes, your trained model will be stored in:`s3://fireguarddata/models/tabnet_balanced/`


## 1. SageMaker Python SDK Training Job for train_tabnet.py 
Create the Training Job Using SageMaker Python SDK

1. Import necessary libraries.
2. Set up an Estimator object with your training script and S3 paths.
3. Launch the training job.
    - Image URI: Uses a GPU-based PyTorch image for your g5 instance.
    - Instance Type: ml.g5.12xlarge is ideal for deep learning with high GPU resources.
    - Region: Uses the US East (N. Virginia) region for the image URI.
    - Hyperparameters: Set epochs to 10 and batch size to 16384 as you mentioned earlier.

In [6]:
!pip install sagemaker

  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached pathos-0.3.3-py3-none-any.whl.metadata (11 kB)
  Preparing metadata (setup.py) ... done
  Using cached platformdirs-4.3.7-py3-none-any.whl.metadata (11 kB)
  Using cached ppft-1.7.6.9-py3-none-any.whl.metadata (12 kB)
  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
  Using cached pox-0.3.5-py3-none-any.whl.metadata (8.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 6.5 MB/s eta 0:00:0000:0100:01
Using cached platformdirs-4.3.7-py3-none-any.whl (18 kB)
Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
Using cached pathos-0.3.3-py3-none-any.whl (82 kB)
Using cached dill-0.3.9-py3-none-any.whl (119 kB)
Using cached pox-0.3.5-py3-none-any.whl (29 kB)
Using cached ppft-1.7.6.9-py3-none-any.whl (56 kB)
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-no

In [ ]:
# Import Required Libraries and Set Up
import sagemaker
from sagemaker.pytorch import PyTorch
import boto3

# Initialize SageMaker session and role
sagemaker_session = sagemaker.Session()
#role = sagemaker.get_execution_role() use only in sagemaker instance
role = "arn:aws:iam::905418478666:role/service-role/AmazonSageMaker-ExecutionRole-20250317T231487"
region = boto3.Session().region_name

# Image URI for PyTorch training (GPU version)
image_uri = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.13.1-gpu-py39"
bucket = "fireguarddata"
scripts_path = f"s3://{bucket}/scripts/"
output_path = f"s3://{bucket}/models/tabnet-new/"

print(f"Region: {region}")
print(f"Role: {role}")
print(f"Image URI: {image_uri}")

# Set Hyperparameters
hyperparameters = {
    "epochs": 20,
    "batch_size": 2048,
    "train_path": "s3://fireguarddata/data/preprocessed_data/train.csv",
    "val_path": "s3://fireguarddata/data/preprocessed_data/val.csv",
}

# Configure the Estimator for Training Job
estimator = PyTorch(
    entry_point="train_tabnet.py",  # Updated script name
    source_dir="./scripts",        # Directory containing the script
    role=role,
    instance_count=1,
    instance_type="ml.g4dn.8xlarge", # Updated to the cheaper instance
    image_uri=image_uri,
    framework_version="1.13.1",
    py_version="py39",
    output_path=output_path,       # Updated output path
    hyperparameters=hyperparameters,
    sagemaker_session=sagemaker_session
)

# Launch the Training Job
estimator.fit(job_name="pytorch-training-tabnet4")
print("Training job launched with ml.g4dn.8xlarge!")


Region: us-east-1
Role: arn:aws:iam::905418478666:role/service-role/AmazonSageMaker-ExecutionRole-20250317T231487
Image URI: 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.13.1-gpu-py39


[04/02/25 21:52:16] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=949550;file:///home/vanel/anaconda3/envs/myenv/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=466777;file:///home/vanel/anaconda3/envs/myenv/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: pytorch-training-tabnet4              ]8;id=861752;file:///home/vanel/anaconda3/envs/myenv/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=14352;file:///home/vanel/anaconda3/envs/myenv/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-04-03 01:52:16 Starting - Starting the training job...
2025-04-03 01:52:32 Starting - Preparing the instances for training...
2025-04-03 01:53:14 Downloading - Downloading the training image..................
2025-04-03 01:56:05 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.9/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.9/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
2025-04-03 01:56:34,285 sagemaker-training-toolkit INFO    

Step 5: Track the Training Job

You can monitor the job's progress either through the AWS Console under SageMaker > Training Jobs or via this notebook by running:

estimator.latest_training_job.wait(logs="All")

## 2. SageMaker Python SDK Training Job for train_tabnet_balanced.py

In [ ]:
# -- Step 1: Import Required Libraries and Set Up
import sagemaker
from sagemaker.pytorch import PyTorch
import boto3

# Initialize SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()  # Get the current SageMaker role
region = boto3.Session().region_name

# Image URI for PyTorch training (GPU version as per your setup)
image_uri = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.13.1-gpu-py39"
bucket = "fireguarddata"  # Your S3 bucket name
scripts_path = f"s3://{bucket}/scripts/"
output_path = f"s3://{bucket}/models/tabnet/"

print(f"Region: {region}")
print(f"Role: {role}")
print(f"Image URI: {image_uri}")

# -- Step 2: Set Hyperparameters and Configuration -- 
# Hyperparameters for training
hyperparameters = {
    "epochs": 10,
    "batch_size": 2048,
    "train_path": "s3://fireguarddata/data/preprocessed_data/train.csv",
    "val_path": "s3://fireguarddata/data/preprocessed_data/val.csv"
}

# --Step 3: Configure the Estimator for Training Job --
# Initialize the PyTorch estimator for the balanced model
estimator_balanced = PyTorch(
    entry_point="train_tabnet_balanced.py",  # Your balanced training script
    source_dir="./scripts",  # The local directory where your script is present 
    role=role,
    instance_count=1,
    instance_type="ml.g4dn.8xlarge",
    image_uri=image_uri,
    framework_version="1.13.1",
    py_version="py39",
    output_path=output_path,
    hyperparameters=hyperparameters,
    sagemaker_session=sagemaker_session
)


#--Step 4: Launch the Training Job -- 
# Start the training job - job_name="pytorch-training-balanced"
estimator_balanced.fit()
print("Balanced training job launched!")

Region: us-east-1
Role: arn:aws:iam::905418478666:role/service-role/AmazonSageMaker-ExecutionRole-20250317T231487
Image URI: 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.13.1-gpu-py39


[03/30/25 06:45:49] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=557803;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=607076;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=476602;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=47783;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             pytorch-training-2025-03-30-06-45-49-350                                              

2025-03-30 06:45:49 Starting - Starting the training job
........30 06:45:49 Pending - Training job waiting for capacity.
..25-03-30 06:47:20 Downloading - Downloading input data.
..............47:30 Downloading - Downloading the training image.
.bash: cannot set terminal process group (-1): Inappropriate ioctl for device..
bash: no job control in this shell
/opt/conda/lib/python3.9/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.9/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
2025-03-30 06:50:34,712 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.traini

## 2. SageMaker Python SDK Training Job for train_tabnet_focal.py

In [ ]:
# -- Step 1: Import Required Libraries and Set Up
import sagemaker
from sagemaker.pytorch import PyTorch
import boto3

# Initialize SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()  # Get the current SageMaker role
region = boto3.Session().region_name

# Image URI for PyTorch training (GPU version as per your setup)
image_uri = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.13.1-gpu-py39"
bucket = "fireguarddata"  # Your S3 bucket name
scripts_path = f"s3://{bucket}/scripts/"
output_path = f"s3://{bucket}/models/tabnet/"

print(f"Region: {region}")
print(f"Role: {role}")
print(f"Image URI: {image_uri}")

# -- Step 2: Set Hyperparameters and Configuration -- 
# Hyperparameters for training
hyperparameters = {
    "epochs": 5,
    "batch_size": 8192,
    "train_path": "s3://fireguarddata/data/preprocessed_data/train.csv",
    "val_path": "s3://fireguarddata/data/preprocessed_data/val.csv"
}

# Step 3: Configure the Estimator for the Focal Loss Model
estimator_focal = PyTorch(
    entry_point="train_tabnet_focal.py",  # Updated to the focal loss script
    source_dir="./scripts",  # Local path where the script is stored
    role=role,
    instance_count=1,
    instance_type="ml.g5.12xlarge",
    image_uri=image_uri,
    framework_version="1.13.1",
    py_version="py39",
    output_path=output_path,
    hyperparameters=hyperparameters,
    sagemaker_session=sagemaker_session
)


# Step 4: Launch the Focal Loss Training Job
estimator_focal.fit(job_name="pytorch-tabnet-training-focal")
print("Focal loss training job launched!")

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/30/25 16:32:09] INFO     Found credentials from IAM Role:                                   ]8;id=963707;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=307276;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[03/30/25 16:32:12] INFO     Found credentials from IAM Role:                                   ]8;id=258035;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=143870;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=456003;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=253452;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Region: us-east-1
Role: arn:aws:iam::905418478666:role/service-role/AmazonSageMaker-ExecutionRole-20250317T231487
Image URI: 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.13.1-gpu-py39


                    INFO     Found credentials from IAM Role:                                   ]8;id=20390;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=800836;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=468986;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=194706;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: pytorch-tabnet-training-focal         ]8;id=842315;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=573165;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-03-30 16:32:13 Starting - Starting the training job
...........16:32:13 Pending - Training job waiting for capacity.

**Step 7: Access the Trained Model**
After training, the model will be saved in your S3 bucket:

**Key Concepts:**
Training Script (train.py): Contains your training logic.

SageMaker Training Job: Runs your script on AWS infrastructure.

Pre-built AWS Container: We used the AWS PyTorch container to simplify dependency management.

Output: Your trained model is saved to an S3 bucket.

## Finding the PyTorch Image URI
AWS SageMaker provides pre-built Docker images for popular frameworks like PyTorch. The image URI depends on the following: framework(pytorch), version, python version (e.g. py39), instance type (cpu/gpu), region(e.g. us. east)
- Using the SageMaker SDK to get the correct image URI:
- the following snippet in your Jupyter notebook to automatically get the correct image URI:

In [9]:
import sagemaker
from sagemaker.pytorch import PyTorch

# Get the AWS region
region = sagemaker.Session().boto_region_name

# Specify the framework, version, Python version, and instance type
framework_version = "1.13.1"
py_version = "py39"
instance_type = "ml.g4dn.8xlarge"  # GPU instance

# Get the correct image URI
image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=region,
    version=framework_version,
    py_version=py_version,
    instance_type=instance_type,
    image_scope="training"  # Important for training jobs
)

print(f"Image URI: {image_uri}")


[04/02/25 20:12:41] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=720808;file:///home/vanel/anaconda3/envs/myenv/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=717515;file:///home/vanel/anaconda3/envs/myenv/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

Image URI: 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.13.1-gpu-py39
